In [1]:
from keras.models import load_model
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Layer

In [3]:
incep= load_model('/content/drive/MyDrive/CVmodel/incepbest/Incepmodel3.h5')

In [4]:
to_res = (224, 224)
res50 = load_model('/content/drive/MyDrive/CVmodel/Resnet50/res50model2.h5')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train_path='/content/drive/MyDrive/CVmodel/traindata'
valid_path='/content/drive/MyDrive/CVmodel/validation'
test_path = '/content/drive/MyDrive/CVmodel/testset'

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)


test_datagen = ImageDataGenerator(rescale = 1./255)

valid_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
training_set = train_datagen.flow_from_directory(train_path,
                                                target_size = (224, 224),
                                                batch_size=16,
                                                class_mode = 'categorical')

Found 1236 images belonging to 4 classes.


In [9]:
valid_set = valid_datagen.flow_from_directory(valid_path,
                                                target_size = (224, 224),
                                                batch_size=16,                                                
                                                class_mode = 'categorical')

Found 216 images belonging to 4 classes.


In [10]:
test_set = test_datagen.flow_from_directory(test_path,
                                            #target_size = (224, 224),
                                            batch_size=8,
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 47 images belonging to 4 classes.


In [25]:
models = [incep,res50]

In [30]:
def ensemble_predictions(members, testX):
  yhats = [model.predict(testX) for model in members]
  yhats = np.array(yhats)
  # sum across ensemble members
  summed = np.average(yhats, axis=0)
  # argmax across classes
  result = np.argmax(summed, axis=1)
  return result

In [31]:
import numpy as np
index= ensemble_predictions(models,test_set)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [32]:
index

array([1, 1, 0, 1, 2, 1, 3, 3, 1, 3, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 3, 2, 2, 1, 2, 3, 3, 3, 1, 3, 2, 3, 3, 1, 3, 3, 1, 2, 1,
       3, 3, 3])

In [33]:
from sklearn.metrics import classification_report
target_names = ['low', 'medium','no_damage','severe']
print(classification_report(test_set.classes,index, target_names=target_names))

              precision    recall  f1-score   support

         low       1.00      0.33      0.50         3
      medium       0.50      0.62      0.55        13
   no_damage       0.73      0.79      0.76        14
      severe       0.73      0.65      0.69        17

    accuracy                           0.66        47
   macro avg       0.74      0.60      0.62        47
weighted avg       0.69      0.66      0.66        47

